In [271]:
import numpy as np
import cv2
import os
from OneNN import PM1NN

In [270]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
def import_data_weiz_reg(numFrames=35,height=50,width=50):
    """
    Collects data from /data/ folder in current working directory, with folder names as labels.
    Inputs:
    -- numFrames=25
    -- height=20,
    -- width=20
    Outputs:
    -- Data as a numpy array (vid number,frame,height,width,color)
    -- Ordered list of video labels
    """
    dataDir = os.getcwd()+"/Weizmann rebuild/"
    dataNameList = os.listdir(dataDir)
    fileCount = 0
    totalFiles = 0
    
    
        
    class_data = []
    labelList = []
    peopleOrder = []
    for foldername in os.listdir(dataDir):
        totalFiles = np.size(os.listdir(dataDir+foldername))
        bigBuffer = np.zeros((totalFiles,numFrames,height,width))
        label = []
        peopleOrder.append(foldername)
        fileCount = 0
        for filename in os.listdir(dataDir+foldername):
            
            vid = cv2.VideoCapture(dataDir+foldername+"/"+filename)
            vidFrames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
            buffer = np.zeros((numFrames,height,width))
            
            for i in range(numFrames):
                sane, thisFrame = vid.read()                
                if not sane:
                    vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
                    sane, thisFrame = vid.read()
                    if not sane:
                        print("something broke at",i,filename)
                        break
                
                scale = 1
                height2, width2, channels = thisFrame.shape
                centerX,centerY=int(height2/2),int(width2/2)
                radiusX,radiusY = int(centerX*scale), int(centerY*scale)
                minX,maxX=centerX-radiusX,centerX+radiusX
                minY,maxY=centerY-radiusY,centerY+radiusY
                cropped = thisFrame[minX:maxX, minY:maxY]
                
                buffer[i] = cv2.cvtColor(cv2.resize(cropped,(width,height)),cv2.COLOR_BGR2GRAY)
                
            label.append(filename)
            bigBuffer[fileCount] = buffer
            fileCount += 1
            vid.release()
        class_data.append(bigBuffer)
        labelList.append(label)
        
    return class_data, labelList, peopleOrder

data, labels, people = import_data_weiz_reg()

In [113]:
def data_pooling(iset):
    
    x,y = [],[]
    
    for curr_class in range(len(iset)):

        x.append(iset[curr_class])
        y.append(curr_class)
            
    return x,y

In [273]:
def data_preprocessing(a,train_index):
    
    testing = a[train_index]
    training = a[:train_index] + a[train_index+1:]
    
    X_test, y_test = data_pooling(testing)
    
    X_train,y_train = [],[]
    
    for i in range(len(training)):
        x,y = data_pooling(training[i])
        X_train.append(x)
        y_train.append(y)
    
    return X_train,y_train,X_test,y_test

In [332]:
X_train,y_train,X_test,y_test = data_preprocessing(data,2)

In [333]:
def compute_accuracy(pred,y_test):
    assert len(pred) == len(y_test)
    return (np.sum(np.array(pred) == np.array(y_test)) / len(pred)) * 100

In [334]:
classifier = PM1NN()
ss = classifier.fit(X_train,y_train)

In [ ]:
pred = classifier.predict(X_test)
accuracy = compute_accuracy(pred,y_test)

[autoreload of six failed: Traceback (most recent call last):
  File "C:\Users\sahil\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\sahil\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\sahil\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "C:\Users\sahil\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 287, in update_class
    old_obj = getattr(old, key)
  File "C:\Users\sahil\anaconda3\lib\site-packages\six.py", line 98, in __get__
    setattr(obj, self.name, result)  # Invokes __set__.
AttributeError: 'NoneType' object has no attribute 'cStringIO'
]
[autoreload of decorator failed: Traceback (most recent call last):
  File "C:\Users\sahil\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, 

In [336]:
accuracy

80.0